In [65]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [66]:
file=pd.read_csv(r"C:\Users\anupr\Desktop\finale year projects\readys000_data.csv")
df = file.drop(['X'], axis=1)
df = df.rename(columns={"species": "label"})

In [67]:
df.head()

,Experience_Level,Current_Location,Client_JD_ID,Tier,Branch,Graduation_Score,Offered_Reject
0,Intermediate,Pune,Cognitive Backend Engineer,T3,Comp Engg,75.0,Yes
1,Intermediate,Mumbai,Cognitive Backend Engineer,T3,Comp Engg,86.0,No
2,Intermediate,Hyderabad,Cognitive Backend Engineer,T1,Comp Engg,83.0,No
3,Senior,Delhi,Cognitive Backend Engineer,T3,Comp Engg,72.0,No
4,Intermediate,Pune,Cognitive Backend Engineer,T3,Electronics,75.0,Yes


In [68]:
#Assigning levels to the categories
lis = []
for i in range(0, df.shape[1]):
    #print(i)
    if(df.iloc[:,i].dtypes == 'object'):
        df.iloc[:,i] = pd.Categorical(df.iloc[:,i])
        #print(file[[i]])
        df.iloc[:,i] = df.iloc[:,i].cat.codes 
        df.iloc[:,i] = df.iloc[:,i].astype('object')
        
        lis.append(df.columns[i])

In [69]:
cname = ['Graduation_Score']

In [70]:
#Nomalisation
for i in cname:
    
    df[i] = (df[i] - min(df[i]))/(max(df[i]) - min(df[i]))

# Train-Test-Split

In [71]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [72]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

# Helper Functions

In [91]:
data = train_df.values
data[:2]

array([[1, 8, 1, 2, 3, 0.475, 1],
       [1, 7, 1, 2, 3, 0.75, 0]], dtype=object)

# Data pure?

In [74]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [93]:
check_purity(train_df.values)

False

# Classify

In [94]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

# Potential splits?

In [98]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the offer_rejected 
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

# Split Data

In [77]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

#  Entropy

In [78]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [79]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [80]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

# Determine Type of Feature

In [81]:
df.head()

,Experience_Level,Current_Location,Client_JD_ID,Tier,Branch,Graduation_Score,Offered_Reject
0,1,8,1,2,3,0.475,1
1,1,7,1,2,3,0.750,0
2,1,4,1,0,3,0.675,0
3,2,3,1,2,3,0.400,0
4,1,8,1,2,0,0.475,1


In [82]:
example_tree = {"Experience_Level <= 2": ["1", 
                                       {"Experience_Level <= 3": [{"Tier <= 2": ["1", 
                                                                                         "0"]}, 
                                                                   
                                                                "1"]}]}
       

In [83]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

# Algorithm

In [84]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [85]:
tree = decision_tree_algorithm(train_df, max_depth=4)
pprint(tree)

{'Branch = 1': [{'Current_Location = 4': [0,
                                          {'Tier = 2': [{'Experience_Level = 2': [1,
                                                                                  0]},
                                                        0]}]},
                {'Current_Location = 1': [{'Graduation_Score <= 0.4085000000000001': [{'Experience_Level = 3': [1,
                                                                                                                0]},
                                                                                      0]},
                                          {'Graduation_Score <= 0.5425000000000001': [{'Graduation_Score <= 0.5274999999999999': [0,
                                                                                                                                  1]},
                                                                                      0]}]}]}


# Classification

In [86]:
example = test_df.iloc[0]
example

Experience_Level        1
Current_Location        1
Client_JD_ID            2
Tier                    0
Branch                  3
Graduation_Score    0.475
Offered_Reject          0
Name: 394, dtype: object

In [87]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [88]:
classify_example(example, tree)

0

# Calculate Accuracy

In [89]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["Offered_Reject"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [90]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.75